# 使用 K-Means 进行图像颜色分割

**核心概念**: 将图像的每个像素视为 RGB 颜色空间中的一个点，使用 K-Means 聚类将相似颜色归为一类，实现图像颜色压缩与分割

## 应用场景

- **颜色量化**: 将图像压缩到有限的颜色数 (如 GIF 格式最多 256 色)
- **图像分割**: 基于颜色将图像分割成不同区域
- **特征提取**: 提取图像的主色调作为特征
- **图像压缩**: 减少存储空间

## 原理

1. 将图像从 (H, W, 3) 重塑为 (H*W, 3)，每行是一个像素的 RGB 值
2. 使用 K-Means 将像素聚类为 K 个颜色簇
3. 用每个簇的质心颜色替换该簇中所有像素的颜色
4. 重塑回原始图像尺寸

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle

# 设置随机种子
np.random.seed(42)

# 配置 matplotlib
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 加载示例图像

使用 scikit-learn 内置的示例图像，无需外部文件依赖。

In [ ]:
# 加载 sklearn 内置的示例图像 (中国风景图)
china = load_sample_image('china.jpg')

# 图像归一化到 [0, 1] 范围
china = np.array(china, dtype=np.float64) / 255

print(f"原始图像形状: {china.shape}")
print(f"图像尺寸: {china.shape[1]} x {china.shape[0]} 像素")
print(f"颜色通道: {china.shape[2]} (RGB)")
print(f"总像素数: {china.shape[0] * china.shape[1]:,}")
print(f"像素值范围: [{china.min():.2f}, {china.max():.2f}]")

# 显示原始图像
plt.figure(figsize=(10, 6))
plt.imshow(china)
plt.title('原始图像')
plt.axis('off')
plt.tight_layout()
plt.show()

## 2. 数据准备

将图像重塑为适合 K-Means 的格式。

In [ ]:
# 获取图像尺寸
h, w, d = china.shape

# 重塑图像: (H, W, 3) -> (H*W, 3)
# 每行代表一个像素的 RGB 值
X = china.reshape(-1, 3)

print(f"重塑后的数据形状: {X.shape}")
print(f"样本数 (像素数): {X.shape[0]:,}")
print(f"特征数 (RGB): {X.shape[1]}")

## 3. 基础颜色分割

使用 K-Means 将图像颜色量化到 K 种颜色。

In [ ]:
def color_quantization(image, n_colors, method='kmeans'):
    """
    使用聚类进行图像颜色量化
    
    参数:
        image: 输入图像，形状为 (H, W, 3)，值范围 [0, 1]
        n_colors: 量化后的颜色数
        method: 'kmeans' 或 'minibatch'
    
    返回:
        quantized: 量化后的图像
        labels: 每个像素的簇标签
        palette: 颜色调色板 (质心)
    """
    h, w, d = image.shape
    X = image.reshape(-1, 3)
    
    # 选择聚类算法
    if method == 'minibatch':
        # 对于大图像，使用 Mini-Batch K-Means 更高效
        model = MiniBatchKMeans(
            n_clusters=n_colors,
            batch_size=1024,
            n_init=10,
            random_state=42
        )
    else:
        model = KMeans(
            n_clusters=n_colors,
            n_init=10,
            random_state=42
        )
    
    # 聚类
    labels = model.fit_predict(X)
    
    # 获取调色板 (质心颜色)
    palette = model.cluster_centers_
    
    # 用质心颜色替换原像素颜色
    quantized = palette[labels].reshape(h, w, d)
    
    # 确保值在有效范围内
    quantized = np.clip(quantized, 0, 1)
    
    return quantized, labels.reshape(h, w), palette

# 使用 64 种颜色进行量化
n_colors = 64
quantized_64, labels_64, palette_64 = color_quantization(china, n_colors, method='minibatch')

print(f"量化后颜色数: {n_colors}")
print(f"压缩比: {256**3 / n_colors:.0f}x (从 1670 万色到 {n_colors} 色)")

In [ ]:
# 对比原图和量化后的图像
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(china)
axes[0].set_title('原始图像 (约 1670 万色)')
axes[0].axis('off')

axes[1].imshow(quantized_64)
axes[1].set_title(f'量化后图像 ({n_colors} 色)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 4. 不同颜色数的效果对比

探索不同 K 值对图像质量的影响。

In [ ]:
# 测试不同的颜色数
color_counts = [2, 4, 8, 16, 32, 64]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, n_colors in enumerate(color_counts):
    quantized, _, _ = color_quantization(china, n_colors, method='minibatch')
    axes[i].imshow(quantized)
    axes[i].set_title(f'{n_colors} 种颜色')
    axes[i].axis('off')

plt.suptitle('不同颜色数量的图像量化效果', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 颜色调色板可视化

展示 K-Means 提取的主要颜色。

In [ ]:
def visualize_palette(palette, title='颜色调色板'):
    """
    可视化颜色调色板
    """
    n_colors = len(palette)
    # 创建调色板图像
    palette_img = palette.reshape(1, n_colors, 3)
    
    fig, ax = plt.subplots(figsize=(12, 1.5))
    ax.imshow(palette_img, aspect='auto')
    ax.set_title(title)
    ax.set_xticks(range(n_colors))
    ax.set_xticklabels([f'{i+1}' for i in range(n_colors)])
    ax.set_yticks([])
    plt.tight_layout()
    plt.show()

# 提取 8 种主要颜色并可视化
_, _, palette_8 = color_quantization(china, 8, method='minibatch')
visualize_palette(palette_8, '8 种主要颜色')

# 提取 16 种主要颜色
_, _, palette_16 = color_quantization(china, 16, method='minibatch')
visualize_palette(palette_16, '16 种主要颜色')

## 6. 图像分割可视化

展示图像被分割成不同颜色区域的效果。

In [ ]:
# 使用较少颜色进行分割可视化
n_segments = 5
quantized_seg, labels_seg, palette_seg = color_quantization(china, n_segments, method='minibatch')

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 原图
axes[0].imshow(china)
axes[0].set_title('原始图像')
axes[0].axis('off')

# 分割标签
im = axes[1].imshow(labels_seg, cmap='tab10')
axes[1].set_title('分割区域 (簇标签)')
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], shrink=0.6)

# 量化结果
axes[2].imshow(quantized_seg)
axes[2].set_title(f'量化结果 ({n_segments} 色)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# 显示调色板
visualize_palette(palette_seg, f'{n_segments} 种分割颜色')

## 7. 另一个示例: 花朵图像

使用 sklearn 内置的另一张示例图像。

In [ ]:
# 加载花朵图像
flower = load_sample_image('flower.jpg')
flower = np.array(flower, dtype=np.float64) / 255

print(f"花朵图像形状: {flower.shape}")

# 不同颜色数量的量化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 原图
axes[0, 0].imshow(flower)
axes[0, 0].set_title('原始图像')
axes[0, 0].axis('off')

# 不同 K 值
for i, n_colors in enumerate([4, 8, 16, 32, 64]):
    row, col = divmod(i + 1, 3)
    quantized, _, _ = color_quantization(flower, n_colors, method='minibatch')
    axes[row, col].imshow(quantized)
    axes[row, col].set_title(f'{n_colors} 种颜色')
    axes[row, col].axis('off')

plt.suptitle('花朵图像颜色量化', fontsize=14)
plt.tight_layout()
plt.show()

## 8. 量化质量评估

使用均方误差 (MSE) 和峰值信噪比 (PSNR) 评估图像量化质量。

In [ ]:
def evaluate_quantization(original, quantized):
    """
    评估图像量化质量
    
    返回:
        mse: 均方误差
        psnr: 峰值信噪比 (dB)
    """
    mse = np.mean((original - quantized) ** 2)
    if mse == 0:
        psnr = float('inf')
    else:
        psnr = 10 * np.log10(1.0 / mse)  # 假设最大值为 1
    return mse, psnr

# 评估不同颜色数的量化质量
color_counts = [2, 4, 8, 16, 32, 64, 128, 256]
mse_values = []
psnr_values = []

for n_colors in color_counts:
    quantized, _, _ = color_quantization(china, n_colors, method='minibatch')
    mse, psnr = evaluate_quantization(china, quantized)
    mse_values.append(mse)
    psnr_values.append(psnr)
    print(f"颜色数: {n_colors:3d}, MSE: {mse:.6f}, PSNR: {psnr:.2f} dB")

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].semilogx(color_counts, mse_values, 'bo-', linewidth=2, markersize=8, base=2)
axes[0].set_xlabel('颜色数量', fontsize=12)
axes[0].set_ylabel('均方误差 (MSE)', fontsize=12)
axes[0].set_title('颜色数量 vs MSE')
axes[0].grid(True, alpha=0.3)

axes[1].semilogx(color_counts, psnr_values, 'ro-', linewidth=2, markersize=8, base=2)
axes[1].set_xlabel('颜色数量', fontsize=12)
axes[1].set_ylabel('PSNR (dB)', fontsize=12)
axes[1].set_title('颜色数量 vs PSNR')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 总结

### 关键要点

1. **原理**: 将图像像素作为 RGB 空间中的点进行聚类，用质心颜色替换簇内所有像素

2. **颜色数选择**:
   - 2-8 色: 艺术风格化效果，失真明显
   - 16-32 色: 适合图标、简单图形
   - 64-256 色: 接近原图质量，适合实际应用

3. **算法选择**:
   - 小图像 (< 100x100): 标准 K-Means
   - 大图像: Mini-Batch K-Means (更高效)

### 应用场景

- **GIF 制作**: 将真彩色图像转换为 256 色以下
- **图像压缩**: 减少颜色数降低存储空间
- **主色调提取**: 提取图像的代表性颜色
- **图像分割**: 基于颜色分割图像区域
- **风格化处理**: 创建海报化、卡通化效果

### 局限性

- K-Means 基于欧氏距离，在 RGB 空间中可能不符合人眼感知
- 考虑使用 LAB 颜色空间可获得更好的视觉效果
- 对于纹理丰富的图像，可能需要更多颜色数